In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import from_unixtime

In [ ]:
# Spark session & context
spark = (SparkSession
         .builder
         .master('local')
         .appName('gps-consumer')
         .getOrCreate())
sc = spark.sparkContext

In [ ]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "gps") \
  .load()

In [ ]:
decode_value = df[['value']]
decode_value = decode_value.withColumn("value", decode_value.value.cast("String"))

In [ ]:
decode_value = decode_value.withColumn("value_array", split(decode_value.value, ',')).drop(decode_value.value)
gps_df = decode_value.withColumn("timestamp", decode_value.value_array.getItem(0).cast("double").cast("timestamp")).withColumn("latitud", decode_value.value_array.getItem(1)).withColumn("longitud", decode_value.value_array.getItem(2)).withColumn("velocidad", decode_value.value_array.getItem(3)).withColumn("id_vehiculo",decode_value.value_array.getItem(4)).drop(decode_value.value_array)

In [ ]:
gps_df

In [ ]:
# Inicia la consulta e imprime el resultado
query = gps_df \
    .writeStream \
    .format("memory") \
    .queryName("gps_data") \
    .start()

In [ ]:
from IPython.display import display, clear_output
from time import sleep

while True:
    clear_output(wait=True)
    display(query.status)
    display(spark.sql('SELECT * FROM gps_data ORDER BY timestamp DESC').show(truncate=False))
    sleep(1)